In [5]:
#Imports
import torch
import numpy as np 
import time # For meassuring running time of the training.

import pandas as pd
import matplotlib.pyplot as plt
import json
import os
from os import listdir

from PIL import Image
import seaborn as sns
sns.set()

# ResNet Model and it's supporting functions
from ResNET import Block, ResNET
from helper_functions import to_categorical, calculate_accuracy, prepare_data_for_resnet


# For tensorflow related imports
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import plot_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# For inception
from tensorflow.keras.applications.inception_v3 import InceptionV3
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization, Conv2D, MaxPool2D, MaxPooling2D
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
from tensorflow.keras.models import load_model

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix


# Disabling all the warnings in the final version
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

ModuleNotFoundError: No module named 'tensorflow'

## ResNet Implementation

#### Generating data for ResNET:

In [ ]:
#base_path = "./data/"
dataset_path = "./data/"#base_path + "/dataset/"

path = './data/**/*.png'

data_size = 5
batch_size = 2
target_size = (224,224)

train_loader, valid_loader, test_loader = prepare_data_for_resnet(path, data_size, batch_size)

### Configurations

In [ ]:
#Configurations

num_epochs = 1
lr = 0.001
batch_size = 128
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

num_classes = 2
img_channel = 3

__Assert block__ to check if the implemented Model class return the correct label dim

In [ ]:
def test():
    resnet = ResNET(Block, [3, 4, 6, 3], img_channel, num_classes)
    _x = torch.randn(2, 3, 244, 244)
    _y = resnet(_x).to(device)
    return (_y.shape)
assert test() == (2,2), “Y is wrong shape!”
print(“Correct!“)

#### Initializing the ResNet Model

In [ ]:
# ResNET Layers [3, 4, 6, 3] for 34 and 50
model = ResNET(Block, [3, 4, 6, 3], img_channel, num_classes)
model.to(device)

loss = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

#### Train the model!

In [ ]:
start_time = time.time()
minibatch_loss_list, train_acc_list, valid_acc_list = [], [], []

for epoch in range(num_epochs):
    model = model.train()
    for batch_idx, (features, targets) in enumerate(train_loader):
        
        features = features.to(device)
        targets = targets.to(device)
      
        features = torch.tensor(features, dtype=torch.float32)
        features = torch.transpose(features, 3, 1)#(x, 1, 3)
        targets = torch.tensor(targets, dtype=torch.float32)
      
        logits = model(features)
        cost = loss(logits, targets)
        optimizer.zero_grad()
        cost.backward()

        optimizer.step()
      
        if not batch_idx % 50:
            print ('Epoch: %03d/%03d | Batch %03d/%03d | Cost: %.4f' 
                    %(epoch+1, num_epochs, batch_idx, 
                      len(train_loader), cost))
          
    model = model.eval() # eval mode to prevent upd. batchnorm params during inference
    with torch.set_grad_enabled(False):# save memory during inference
        train_acc, train_pred = calculate_accuracy(model, train_loader)
        valid_acc, valid_pred = calculate_accuracy(model, valid_loader)
        print('Epoch: %03d/%03d training accuracy: %.2f%% | Validation accuracy: %.2f%%' % (
              epoch+1, num_epochs, 
              train_acc, valid_acc))
        train_acc_list.append(train_acc.item())
        valid_acc_list.append(valid_acc.item())

    print('Time elapsed: %.2f min' % ((time.time() - start_time)/60))

print('Total Training Time: %.2f min' % ((time.time() - start_time)/60))

In [ ]:
# Saving the model for future use
#torch.save(model.state_dict(), 'resNET_model.pth')

#### Display the accuracy Plot:

In [ ]:
num_epochs = len(train_acc_list)

plt.plot(np.arange(1, num_epochs+1),
          train_acc_list, label='Training')
plt.plot(np.arange(1, num_epochs+1),
          valid_acc_list, label='Validation')

plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()

#### Computing test accuracy and plotting a few examples with their predictions

In [ ]:
acc, predictions = calculate_accuracy(model, test_loader)
print('Test accuracy: %.2f%%' % (acc))

#%% Training data visualization
images, labels = next(iter(test_loader))

plt.figure(figsize=(5,5))
for i in range(9):
    
    plt.subplot(3, 3, i+1)
    label = np.argmax( labels[i+10] ,axis=0)
    
    ## Debugging
    #print(len(label))
    #print(predictions.shape)
    #print(images[i].shape, ' : ', images[i].permute(1, 2, 0).shape)
    #print(predictions[i].item(), '|', label.item())
    
    str = F"P: {predictions[i+10].item()},\nL: {label.item()}"
    plt.title(str)
    plt.imshow(images[i+10])
    
    plt.axis('off')

plt.tight_layout()
plt.show()

In [ ]:

def get_predictions_and_labels(data_loader, device = 'cpu'):
    
    labels = None
    logits = None
    
    for batch_idx, (features, targets) in enumerate(data_loader):
        features = torch.tensor(features, dtype=torch.float32)
        features = torch.transpose(features, 3, 1)#(x, 1, 3)
        targets = torch.tensor(targets, dtype=torch.float32)
        
        features = features.to(device)
        targets = targets.to(device)
        
        logits = model(features)
        labels = targets
    
    labels = np.argmax( labels ,axis=1)
    logits = np.argmax( logits.detach().numpy() ,axis=1)
    
    return labels, logits

def get_classification_matrics(data_loader, target_names = ['0', '1'], device = 'cpu'):
    labels, prediction = get_predictions_and_labels(data_loader, device)
    
    print(classification_report(labels, prediction, target_names=target_names))
    
    print_confusion_matrix(labels, prediction, 
                           title = 'Confusion Matrix of ResNET',
                           x_lable = 'Predicted Label',
                           y_lable = 'True Label')
    
def print_confusion_matrix(labels, prediction, title, x_lable, y_lable, fig_size = 5):
    
    resNet_cm = confusion_matrix(labels, prediction)
    f,ax = plt.subplots(figsize=(fig_size, fig_size))
    sns.heatmap(resNet_cm, annot=True, linewidths=0.01,cmap="OrRd",linecolor="black", fmt= '.1f',ax=ax)
    plt.xlabel(y_lable)
    plt.ylabel(x_lable)
    plt.title(title)
    plt.show()
    

get_classification_matrics(test_loader)

### Importing Inception

In [ ]:
folder = listdir(base_path)
#len(folder)

total_images = 0
for n in range(len(folder)):
    patient_id = folder[n]
    for c in [0, 1]:
        patient_path = base_path + patient_id 
        class_path = patient_path + "/" + str(c) + "/"
        subfiles = listdir(class_path)
        total_images += len(subfiles)
        
#total_images

In [ ]:
# Comment

In [ ]:
data = pd.DataFrame(index=np.arange(0, total_images), columns=["patient_id", "path", "target"])

k = 0
for n in range(len(folder)):
    patient_id = folder[n]
    patient_path = base_path + patient_id 
    for c in [0,1]:
        class_path = patient_path + "/" + str(c) + "/"
        subfiles = listdir(class_path)
        for m in range(len(subfiles)):
            image_path = subfiles[m]
            data.iloc[k]["path"] = class_path + image_path
            data.iloc[k]["target"] = c
            data.iloc[k]["patient_id"] = patient_id
            k += 1  

data.head()

In [ ]:
non_cancer_data = data.query("target == 0").head(data_size)
print(non_cancer_data['target'].value_counts())
cancer_data = data.query("target == 1").head(data_size)
print(cancer_data['target'].value_counts())
sliced_data = pd.concat([non_cancer_data,cancer_data])
print(sliced_data['target'].value_counts())

print(sliced_data['target'].value_counts())

In [ ]:
sliced_data.target.unique()

sliced_data.shape

In [ ]:
sliced_data.head()
sliced_data.loc[:, "target"] = data.target.astype(np.str)
sliced_data.info()

unique_paths = sliced_data.path.unique()

In [ ]:
sub_train_ids, test_ids = train_test_split(unique_paths,
                                           test_size=0.2,
                                           random_state=0)
train_ids, valid_ids = train_test_split(sub_train_ids, test_size=0.1, random_state=0)

print(f"Now, we're taking {round(len(train_ids)/unique_paths.shape[0]*100, 1)}% for training, 
      {round(len(valid_ids)/unique_paths.shape[0]*100,1)}% for validation, 
      {round(len(test_ids)/unique_paths.shape[0]*100,1)}% for testing")

print(f'{len(train_ids)}, {len(valid_ids)}, {len(test_ids)}')


In [ ]:
train_df = sliced_data.loc[sliced_data.path.isin(train_ids),:].copy()
test_df = sliced_data.loc[sliced_data.path.isin(test_ids),:].copy()
valid_df = sliced_data.loc[sliced_data.path.isin(valid_ids),:].copy()

print(f"train set shape: {train_df.shape}")
print(f"test set shape: {test_df.shape}")
print(f"validation set shape: {valid_df.shape}")

In [ ]:
train_df.reset_index(drop=True, inplace=True)
test_df.reset_index(drop=True, inplace=True)
valid_df.reset_index(drop=True, inplace=True)

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   rotation_range = 40, 
                                   width_shift_range = 0.2, 
                                   height_shift_range = 0.2, 
                                   shear_range = 0.2, 
                                   zoom_range = 0.2, 
                                   horizontal_flip = True, 
                                   vertical_flip =True)

test_datagen = ImageDataGenerator(rescale = 1./255)

valid_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
train_batches = train_datagen.flow_from_dataframe(
    train_df,
    x_col = 'path', 
    y_col ='target',
    target_size=target_size,
    batch_size=batch_size,
    class_mode='binary',
    shuffle=True,
)

valid_batches = valid_datagen.flow_from_dataframe(
    valid_df,
    x_col = 'path', 
    y_col ='target',
    target_size=target_size,
    batch_size=batch_size,
    class_mode='binary',
    shuffle=True,
)

test_batches = test_datagen.flow_from_dataframe(
    test_df,
    x_col = 'path', 
    y_col ='target',
    target_size=target_size,
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False,
)

In [ ]:
fig, ax = plt.subplots(1,3,figsize=(20,5))
sns.countplot(train_df.target, ax=ax[0], palette="Reds")
ax[0].set_title("Train data")
sns.countplot(valid_df.target, ax=ax[1], palette="Blues")
ax[1].set_title("Valid data")
sns.countplot(test_df.target, ax=ax[2], palette="Greens");
ax[2].set_title("Test data");

In [ ]:
def InceptionV3(base_model,save_model_path):
    """Explanation:
    
    Arguments
    ---------
    base_model :
    tf.keras.applications.InceptionV3
    --------
    save_model_path : containing the path where to save the model.
    
    Returns
    ---------
    InceptionV3_model: keras.engine.functional.Functional
    -------------
    """ 
    """ Creating Model """

    base_model = base_model
    
    # Freezing Layers  
    """..Freezing the layers and making it non-training except the last three layers.."""
    for layer in base_model.layers[:-3]:
        
        layer.trainable=False 

    """ ...Building Inception Model...."""
    InceptionV3_model=Sequential() # Making it sequential 
    InceptionV3_model.add(base_model) # Adding the base_model to the sequencial model

    """Adding the Droupout and Flatten Layers"""
    InceptionV3_model.add(Dropout(0.5))
    InceptionV3_model.add(Flatten())

    """Adding Dense layer and sigmoid function"""
    InceptionV3_model.add(Dense(1,activation='sigmoid',name='output'))

    return InceptionV3_model

"""Call this function to create inceptionV3 model..."""        
InceptionV3_model = InceptionV3(base_model=tf.keras.applications.InceptionV3(input_shape=(224,224,3),
                                                                             include_top=False,
                                                                             weights="imagenet"),
                                save_model_path="./test/")

InceptionV3_model.summary() # Summary of the created model.

In [ ]:
def training_loop(InceptionV3_model,train_batches, valid_batches, epochs):
    """Explanation:
    
    Arguments
    ---------
    InceptionV3_model :
    
    containing implemented model
    
    --------
    
    train_batches :
    containing the training for training the inceptionV3 model.

    ---------
    
    valid_batches :
    containing the validation set required for observing the hyperparameters of the inceptionV3 model.
    
    ---------
    epochs : number of epochs needed for training
    
    Returns : N/A
    -------------
    """ 
    
    """ Crompiling the Model """
    
    InceptionV3_model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(learning_rate= 0.0001), metrics=['accuracy'])
    
    
    """ Checkpoints for monitoring validation accuracy by which it helps the model to save when there is no improvement. """"
    
    model_checkpoints = tf.keras.callbacks.ModelCheckpoint("Breast_Cancer_InceptionV3_model.weights.best.h5", save_best_only=True, verbose = 1,  mode='max',  monitor='val_accuracy')
    
    """" It helps the model to save from under and over fitting. It saves the model there is no improvement anymore to save the model from overfitting."""
    early_stopping = keras.callbacks.EarlyStopping( monitor="val_accuracy", mode="max",patience=5,verbose=1)
    
    # Regarding Early Stop and Model Checkpoints as Calssbacks
    callbacks = [model_checkpoints,early_stopping]
    
    """Saving the history of the model.."""
    inception_history = InceptionV3_model.fit(train_batches, validation_data = valid_batches, epochs = num_epochs, callbacks = [callbacks], verbose = 1)
    
training_loop(InceptionV3_model,train_batches, valid_batches, num_epochs)

In [ ]:
"""Loading the already trained model..."""
def load_InceptionV3(loaded_model):
    model=load_model(loaded_model)
    return model
loaded_InceptionV3 = load_InceptionV3("inceptionv3_mode.h5")

In [ ]:
def print_test_inceptionV3(test_batches):
    inception_model_loss, inception_model_accuracy = InceptionV3_model.evaluate(test_batches, verbose=2)
    print("Restored model, accuracy: {:5.2f}%".format(100 * inception_model_accuracy))
    print("Restored model, loss: {:5.2f}%".format(100 * inception_model_loss))
    
print_test(test_batches)

In [ ]:
#Loss
plt.plot(inception_history.history['loss'], label='Train Loss')
plt.plot(inception_history.history['val_loss'], label='Validation Loss')
plt.title("InceptionV3 Model Loss")
plt.ylabel("Loss")
plt.xlabel("Epoch")
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

In [ ]:
#Accuracies
plt.plot(inception_history.history['accuracy'], label='Train Accuracy')
plt.plot(inception_history.history['val_accuracy'], label='Validation Accuracy')
plt.title("InceptionV3 Model Accuracy")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
plt.legend()
plt.show()
plt.savefig('AccVal_acc')

In [ ]:
def scores() :
    
    """Doining prediction for each classes."""
    
    Y_pred_inception = InceptionV3_model.predict_generator(test_batches, test_df.shape[0] // batch_size+1)
    y_pred_inception = np.where(Y_pred_inception > 0.5, 1, 0).flatten()
    
    """Doining prediction for each classes."""
    
    inception_cm = confusion_matrix(test_batches.classes, y_pred_inception)
    f,ax = plt.subplots(figsize=(8, 8))
    sns.heatmap(inception_cm, annot=True, linewidths=0.01,cmap="Purples",linecolor="black", fmt= '.1f',ax=ax)
    
    """Plotting confusion matrix title."""
    
    plt.xlabel("Predicted Label")
    plt.ylabel("True Label")
    plt.title("Confusion Matrix of InceptionV3")
    plt.show()
    
    """Plotting score values : Precision, Recall, f1-score"""
    
    print(classification_report(test_batches.classes, y_pred_inception, target_names=['0', '1']))
    
scores()

In [ ]:
# VGG

## VGG

### Model Creation

The following functions will be useful to create a VGG16 model. The function  `create_vgg16_model` will create/load a VGG16 model.


**Run the code cell below** to create a model. 

In [ ]:

# VGG16
vgg16_model_path = "VGG16_model.h5"
vgg16_model_plot_path = "vgg16_model.png"
vgg16_model_history_path = "vgg16_history.json"

In [ ]:
def create_vgg16_model(model_path=None):
    """breif description.
    
    Arguments
    ---------
    model_path : String
      String containing the path of a model.
    
    Returns
    ---------
    vgg_model: keras.engine.functional.Functional
      Model containing .... 
    """
    if model_path:
        return tf.keras.models.load_model(model_path)
    else:
        """ Load VGG model with imagenet trained weights and pass our input shape . """
        base_model = tf.keras.applications.vgg16.VGG16(
            weights='imagenet', # Load weights pre-trained on ImageNet.
            input_shape=(224, 224, 3),
        )
        
        """ Make loaded layers as non-trainable. This is important as we want to work with pre-trained weights. """
        for layer in base_model.layers:
            layer.trainable = False
        
        """ Add all layers to model except the output layer. """
        new_model = keras.models.Sequential()
        new_model._name="VGG16"
        
        for layer in base_model.layers[0:-1]:
            new_model.add(layer)
            
        """ Add all layers to model except the output layer. """
        new_model.add(layers.Dense(1, activation="sigmoid", name='output'))
        
        return new_model
        
vgg_model = create_vgg16_model(vgg16_model_path)

### Model Summary

**Run the code cell below** to create a summary of our model. 

In [ ]:
print(type(vgg_model))
vgg_model.summary()

### Model Plotting
**Run the code cell below** to create a plot of a model and download it. 

*Make sure you have installed `pydot` & `graphviz` for plot_model.*

In [ ]:
plot_model(vgg_model, to_file=vgg16_model_plot_path, show_shapes=True, dpi=100)

### Training

**Compile the model with necessary configs:**

* Here I will be using **Adam** optimiser to reach to the global minima while training out model. If I am stuck in local minima while training then the adam optimiser will help us to get out of local minima and reach global minima. We will also specify the **learning rate** of the optimiser, here in this case it is set at **0.001**. If our training is bouncing a lot on epochs then we need to decrease the learning rate so that we can reach global minima.

* **binary_crossentropy** function computes the cross-entropy loss between true labels and predicted labels.

* And metrics will be **accuracy**.


***If you don't want to train the model, you can skip this cell.***

In [ ]:
vgg_model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(learning_rate= 0.0001), metrics=['accuracy'])

**Train the model with callbacks function:**

* **ModelCheckpoint** - It helps us to save the model by monitoring a specific parameter of the model. In this case I am monitoring validation accuracy by passing `val_accuracy` to ModelCheckpoint. The model will only be saved to disk if the validation accuracy of the model in current epoch is greater than what it was in the last epoch.

* **EarlyStopping** - It helps us to stop the training of the model early if there is no increase in the parameter which I have set to monitor in EarlyStopping. In this case I am monitoring validation accuracy by passing `val_accuracy` to EarlyStopping. I have here set patience to `5` which means that the model will stop to train if it doesn’t see any rise in validation accuracy in 5 epochs.


***If you don't want to train the model, you can skip this cell.***

In [ ]:
model_checkpoints = tf.keras.callbacks.ModelCheckpoint(
    "Breast_Cancer_VGG16_model.weights.best.h5", 
    save_best_only=True, 
    verbose=1, 
    mode='max', 
    monitor='val_accuracy'
)
early_stopping = keras.callbacks.EarlyStopping(
    monitor="val_accuracy",
    mode="max",
    patience=5,
    verbose=1
)

callbacks = [
    model_checkpoints,
    early_stopping
]

**Model fitting**

Here, I will pass training and validation data, `epochs` is set to 100, `callbacks` and `verbose` is 2. 

***If you don't want to train the model, you can skip this cell.***

In [ ]:
vgg_history = vgg_model.fit(
    train_batches, 
    validation_data=valid_batches, 
    epochs = num_epochs,
    callbacks=callbacks, 
    verbose=2
)

**Store training history**

***If you don't want to train the model, you can skip this cell.***

In [ ]:
with open(vgg16_model_history_path, 'w') as file:
    json.dump(vgg_history.history, file)

**Retrive training history**

Loading history from a JSON file.

In [ ]:
with open(vgg16_model_history_path) as file:
    vgg_history_data = json.load(file)

**Plot the training and validation outcome**

In [ ]:
plt.plot(vgg_history_data["accuracy"])
plt.plot(vgg_history_data['val_accuracy'])
plt.plot(vgg_history_data['loss'])
plt.plot(vgg_history_data['val_loss'])
plt.title("VGG16 model's training and validation outcome")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
plt.legend(["Accuracy", "Validation Accuracy", "Loss", "Validation Loss"])
plt.rcParams['axes.facecolor'] = 'white'
plt.rcParams['figure.facecolor'] = 'white'
plt.show()

**Save model**

In [ ]:
vgg_model.save(vgg16_model_path)

### Evalutaion

Evaluate model on test data.

In [ ]:
vgg_model_loss, vgg_model_accuracy = vgg_model.evaluate(test_batches, verbose=2)
print("Accuracy: {:5.2f}%".format(100 * vgg_model_accuracy))
print("Loss: {:5.2f}%".format(100 * vgg_model_loss))

### Confusion Matrix

It visualizes and summarizes the performance of a classification algorithm.

*0 -> Non-IDC*

*1 -> IDC*

In [ ]:
Y_pred_vgg = vgg_model.predict_generator(test_batches, test_df.shape[0] // batch_size+1)
y_pred_vgg = np.where(Y_pred_vgg >= 0.5, 1, 0).flatten()
vgg_cm = confusion_matrix(test_batches.classes, y_pred_vgg)
f,ax = plt.subplots(figsize=(4, 4))
sns.heatmap(vgg_cm, annot=True, linewidths=0.01, cmap="Blues", linecolor="black", fmt='.1f', ax=ax)
plt.xlabel("Predicted values")
plt.ylabel("Actual values")
plt.title("Confusion Matrix of VGG16")
plt.show()

### Classification Report

Here, we will have a performance evaluation metric where we can find **precision**, **recall**, **F1 Score**, and **support** of our trained model.

*0 -> Non-IDC*

*1 -> IDC*

In [ ]:
print(classification_report(test_batches.classes, y_pred_vgg, target_names=['0', '1']))

### Prediction

Load one Non-IDC and IDC picture, do prediction on our trained model.

*0 -> Non-IDC*

*1 -> IDC*

In [ ]:
def prediction_for_vgg16(dataframe):
    plt.figure(figsize=(10,10))
    
    for i, v in enumerate([0, 799]):
        plt.subplot(1,2,i+1)
        image = cv2.imread(dataframe.loc[v, "path"])

        # create input data
        image_fromarray = Image.fromarray(image, 'RGB')
        resize_image = image_fromarray.resize((224, 224))
        expand_input = np.expand_dims(resize_image, axis=0)
        input_data = np.array(expand_input)
        input_data = input_data/255

        # prediction
        prediction = vgg_model.predict(input_data)

        # plot the image
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        plt.imshow(image, cmap=plt.cm.binary)
        plt.title("Actual: %s" % ("Non-IDC" if int(dataframe.loc[v, "target"]) == 0 else "IDC") )
        plt.xlabel("Predict: %s" % ("IDC" if prediction >= 0.5 else "Non-IDC"))
        
    plt.show()

prediction_for_vgg16(test_df)